In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os
import csv


## Open Dataset Files

In [4]:
# load .mat from dataset

filename = r'Data 100hz/BCICIV_1_mat/BCICIV_calib_ds1a.mat'

# get data from .mat
def inputmat(fp):
    """load .mat file and return m as a dict"""
    mat = loadmat(fp, squeeze_me=True)
    td = {}  # create a dict

    # Numpy array of size channel_num * points.
    td['data'] = mat['cnt'].T  # data
    td['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.
    td['ch_names'] = mat['nfo']['clab'][True][0]

    # Position of channels
    
    td['electrode_x'] = mat['nfo']['xpos'][True][0]
    td['electrode_y'] = mat['nfo']['ypos'][True][0]

    #make trials by finding trials and its data
    td['cue'] = mat['mrk']['pos'][True][0] #time of cue
    td['labels'] = mat['mrk']['y'][True][0] #labels of the data
    return td


# filter data using bandpass 7 - 30 Hz
def filter_data(data_dict):
    """filter data from data_dict"""
    from scipy import signal
    data = data_dict['data']
    freq = data_dict['freq']
    # bandpass filter
    b, a = signal.butter(4, [7, 30], btype='bandpass', fs=freq)
    data = signal.filtfilt(b, a, data, axis=1)
    data_dict['data'] = data
    return data_dict


def data_to_csv(data_dict, averaging=False, filename='data.csv'):
    """convert data from data_dict to csv file for one channel if averaging enabled"""
    data = data_dict['data']
    labels = data_dict['labels']
    freq = data_dict['freq']
    cue = data_dict['cue']
    ch_names = data_dict['ch_names']
    if averaging:
        data = np.mean(data, axis=0)
        ch_names = ['Averaged']
        # create a new channel by averaging all the channels
        new_data = np.zeros((data.shape[1], 1))
        for i in range(data.shape[1]):
            new_data[i] = np.mean(data[:, i])
        data_dict['data'] = new_data
        data_dict['ch_names'] = ['Averaged']
    # write to csv
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(ch_names)
        for i in range(data.shape[1]):
            writer.writerow(data[:, i])
    return data_dict

data_dict = inputmat(filename)
os.makedirs('Data', exist_ok=True)
os.makedirs('Data 100hz', exist_ok=True)

data_to_csv(data_dict, averaging=False, filename='Data/data.csv')
data_to_csv(filter_data(data_dict), averaging=False, filename='Data/data_filtered.csv')

{'data': array([[-1.14063556e-01, -4.68590380e+01, -4.40241788e+01, ...,
          4.65349620e+01,  1.75819863e+01,  3.99631485e-02],
        [ 2.14469240e-01, -3.81930193e+01, -1.04833541e+01, ...,
          6.47883863e+01,  1.40264717e+01,  3.84727469e-01],
        [-9.64727164e-01, -4.30903673e+01, -5.56090592e+01, ...,
          3.81116916e+01, -9.04713334e-01, -3.07303900e-01],
        ...,
        [-4.38583797e-01,  2.50739572e+01,  4.36944562e+01, ...,
          4.03594194e+01, -1.44868797e+01, -2.98759888e+00],
        [-4.32091980e-01,  5.35999385e+01,  8.24078656e+01, ...,
          3.67810405e+01, -1.93934954e+01, -3.61407778e+00],
        [-2.18449778e-01,  4.26703036e+01,  6.60226333e+01, ...,
          1.28132393e+02,  3.59284892e+01, -3.84965118e+00]]),
 'freq': 100,
 'ch_names': array(['AF3', 'AF4', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'FC5',
        'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'CFC7', 'CFC5', 'CFC3',
        'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T